In [203]:
import pandas as pd
import numpy as np
portfolio = pd.read_csv("portfolio.csv").drop("Unnamed: 0", axis=1)
portfolio.rename({"id": "offer_id"}, axis=1, inplace=True)
pd.get_dummies(portfolio)
d_portfolio = pd.get_dummies(portfolio[portfolio.columns[:5]])
d_portfolio["id"] = portfolio["offer_id"]
# portfolio
portfolio

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"['web', 'email', 'mobile', 'social']",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"['web', 'email', 'mobile', 'social']",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"['email', 'mobile', 'social']",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"['web', 'email', 'mobile', 'social']",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"['web', 'email', 'mobile']",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [218]:
profile = pd.read_csv("profile.csv").drop("Unnamed: 0", axis=1)
profile.rename({"id": "person"}, axis=1, inplace=True)
profile["age"] = profile["age"].replace(118, np.nan)
profile["income"].isna().mean(), profile["gender"].isna().mean(), (profile["age"]==118).mean()
print(((profile["income"].isna() == profile["gender"].isna()) == (profile["age"].isna() == profile["gender"].isna())).mean())
profile = profile[profile["income"].notna()]


profile["became_member_on"] = pd.to_datetime(profile["became_member_on"], format="%Y%m%d")

profile['days_diff'] = (profile["became_member_on"].min() - profile['became_member_on']).dt.days

annual_growth_rate = 0.08
daily_growth_rate = (1 + annual_growth_rate) ** (1/365) 

profile["min_date_income"] = profile['income'] * (daily_growth_rate) ** profile['days_diff']


1.0


In [219]:
max_time_diff = (profile["became_member_on"].max() - profile['became_member_on'].min()).days
profile["max_date_income"] = profile["min_date_income"] * (daily_growth_rate) ** max_time_diff
profile.drop(["income", "min_date_income"], axis=1, inplace=True)
profile["days_diff"] = -profile["days_diff"] 


In [205]:
transcript = pd.read_csv("transcript.csv").drop("Unnamed: 0", axis=1)
# transcript.merge(profile, "inner", "person")
import ast
transcript["value"] = transcript["value"].apply(lambda x: ast.literal_eval(x))

In [131]:
new_transcript = []
for idx,i in transcript.iterrows():
    for key,value in i.value.items():
        new_i = i.copy()
        if key=="offer id":
            key="offer_id"
        
        new_i[key] = value
        new_i.value = key
        new_transcript += [new_i]
        
new_transcript = pd.DataFrame(new_transcript)

In [220]:
new_transcript.to_csv("new_transcript.csv", index=None)
mix = new_transcript.merge(profile, "inner", "person")
mix.drop_duplicates(inplace=True)
# mix.groupby(["person"])["event"].count()

In [221]:
transcript_grouped = mix.groupby("person")
# (transcript_grouped["offer_id"].count()==0).sum()/len(profile), (transcript_grouped["amount"].count()==0).sum()/len(profile), (transcript_grouped["reward"].count()==0).sum()/len(profile)
# Offer ID and amount discounts are given to almonst everyone
hoho = profile.set_index("person")

hoho["offer_id_count"] = transcript_grouped["offer_id"].count()
hoho["amount_count"] = transcript_grouped["amount"].count()
hoho["amount_mean"] = transcript_grouped["amount"].mean()
hoho["reward_count"] = transcript_grouped["reward"].count()
hoho["reward_mean"] = transcript_grouped["reward"].mean()

In [222]:
new_profile = hoho.reset_index()
new_profile.to_csv("new_profile.csv", index=None)

In [223]:
mix.merge(portfolio, "inner", "offer_id")

,person,event,value,time,offer_id,amount,reward_x,gender,age,became_member_on,days_diff,max_date_income,reward_y,channels,difficulty,duration,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,offer_id,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,F,75.0,2017-05-09,1380,109790.905326,5,"['web', 'email', 'mobile']",5,7,bogo
1,e2127556f4f64592b11af22de27a7932,offer received,offer_id,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,M,68.0,2018-04-26,1732,71356.096788,2,"['web', 'email', 'mobile']",10,7,discount
2,389bc3fa690240e798340f5a15918d5c,offer received,offer_id,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,M,65.0,2018-02-09,1656,54899.499150,5,"['web', 'email', 'mobile', 'social']",5,5,bogo
3,2eeac8d8feae4a8cad5a6af0499a211d,offer received,offer_id,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,M,58.0,2017-11-11,1566,53839.889867,0,"['web', 'email', 'mobile']",0,4,informational
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,offer_id,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,F,61.0,2017-09-11,1505,60952.950123,5,"['web', 'email']",20,10,discount
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148426,84fb57a7fe8045a8bf6236738ee73a0f,offer viewed,offer_id,714,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,F,64.0,2017-06-15,1417,123099.601837,0,"['email', 'mobile', 'social']",0,3,informational
148427,abc4359eb34e4e2ca2349da2ddf771b6,offer viewed,offer_id,714,3f207df678b143eea3cee63160fa8bed,NaN,NaN,M,51.0,2014-08-04,371,92357.327869,0,"['web', 'email', 'mobile']",0,4,informational
148428,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,offer_id,714,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,F,60.0,2017-09-08,1502,68481.704988,5,"['web', 'email']",20,10,discount
148429,8431c16f8e1d440880db371a68f82dd0,offer completed,offer_id,714,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,M,39.0,2018-06-27,1794,39239.204389,2,"['web', 'email', 'mobile', 'social']",10,10,discount


In [224]:
portfolio

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"['web', 'email', 'mobile', 'social']",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"['web', 'email', 'mobile', 'social']",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"['email', 'mobile', 'social']",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"['web', 'email', 'mobile', 'social']",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"['web', 'email', 'mobile']",10,7,discount,2906b810c7d4411798c6938adc9daaa5
